# Memahami TensorFlow: Eager Mode, Graph Mode, dan AutoGraph

Jika Anda menggunakan **TensorFlow 2**, khususnya jika Anda mempelajarinya menggunakan **myCourses**, Anda akan menggunakan **high-level APIs** dan **Keras**, serta **Eager mode** untuk mempermudah pengembangan dan debugging. Namun terkadang, Anda mungkin ingin memperoleh performa ekstra dari kode Anda, dan salah satu cara untuk melakukannya adalah dengan menggunakan **graph-based models**.

### Eager Mode vs Graph Mode
Pada minggu ini, Anda akan mempelajari **AutoGraph technology** yang mempermudah pengembangan kode berbasis graph. Sebelumnya, Anda sudah melihat bagaimana **Eager mode** dapat mempermudah penulisan kode yang memberikan hasil langsung. 

**TensorFlow** awalnya dirancang di sekitar pemrograman dalam **graph mode**, di mana Anda harus mendefinisikan grafik dengan semua operasi sebelum mengeksekusinya. Misalnya, jika Anda ingin menghitung rumus seperti `ReLU(y) = ReLU(Wx + b)`, Anda akan memiliki grafik seperti ini:

- Anda akan memperlakukan **W** dan **X** sebagai variabel yang dimasukkan ke dalam operasi perkalian (atau **op** untuk singkatannya).
- Hasil perkalian ini akan dimasukkan ke dalam operasi penjumlahan, bersama dengan variabel **b**.
- Hasil penjumlahan ini kemudian dimasukkan ke dalam operasi lain, seperti **ReLU**, untuk menghasilkan jawaban **y**.

Meskipun cara ini mungkin tidak langsung intuitif bagi Anda sebagai pengembang Python, operasi berbasis grafik dapat berjalan sangat cepat dan menggunakan grafik dapat mempercepat **training** dan **inference**. Namun, penulisan kode berbasis grafik bisa menjadi lebih rumit dan sulit untuk **debug** karena operasi seperti perkalian, penjumlahan, dan ReLU baru akan dilakukan setelah grafik selesai dirancang.

### Control Flow dalam Graph Mode
Lebih jauh lagi, mari kita pertimbangkan **control flow**, seperti fungsi dengan pernyataan **"if"**. Misalnya, jika **x** lebih besar dari nol, kembalikan **x** kuadrat, jika tidak kembalikan **x**.

- Dalam **Eager execution** di Python, Anda dapat menulis kode dengan kontrol alur yang sangat sederhana dan familiar.
- Tetapi dalam **graph mode**, Anda tidak dapat menggunakan kondisi **if** langsung. Anda harus menulis kode seperti ini menggunakan **tf.cond** untuk kondisional. Di sini, Anda bisa membandingkan apakah **x** lebih besar dari nol menggunakan **tf.greater**, dengan parameter berikutnya adalah fungsi yang dipanggil jika benar (**if_true**), dan parameter setelah itu adalah fungsi yang dipanggil jika salah (**if_false**).

### Keuntungan dan Keterbatasan Mode Eager dan Graph
- **Eager mode** memungkinkan Anda menulis kode Python standar dengan sintaks kontrol alur standar, tetapi Anda kehilangan beberapa manfaat dari grafik.
- Dalam **AutoGraph**, grafik memiliki **dependencies eksplisit**. Maksudnya, jika Anda melihat sebuah node dalam grafik, Anda dapat mengetahui operasi apa yang harus dijalankan sebelumnya dengan melacaknya secara mundur di grafik. 

Mengetahui dependensi untuk setiap operasi memungkinkan Anda untuk mencari efisiensi, seperti menjalankan beberapa operasi secara paralel atau mendistribusikan operasi ke mesin yang berbeda.

### Menggunakan Kedua Pendekatan: Eager dan Graph Mode
Meskipun **Eager mode** dan **graph mode** tampaknya saling bertentangan, kita bisa memanfaatkan keduanya. Sebagai contoh, Anda dapat menggunakan **Eager mode** saat mengembangkan model baru atau melakukan debugging, dan kemudian beralih ke **graph mode** jika Anda ingin meningkatkan performa model.

### Apa Itu AutoGraph?
**AutoGraph** adalah teknologi yang memungkinkan Anda mengubah kode Pythonik gaya Eager Anda secara otomatis menjadi grafik, dan sebaliknya. Ini memberikan keuntungan dengan mempermudah proses konversi antara kedua mode tersebut, memungkinkan Anda mendapatkan yang terbaik dari kedua dunia.

Dengan AutoGraph, Anda bisa mengembangkan model menggunakan gaya **Eager** yang mudah digunakan, lalu mengkonversinya ke **graph mode** untuk memperoleh performa yang lebih optimal.


# Penjelasan tentang `tf.function` dan Autograph pada TensorFlow

Pada video sebelumnya, Anda telah diperkenalkan dengan **autograph** dan bagaimana dengan menghias fungsi Anda menggunakan `tf.function`, TensorFlow dapat membuat kode **graph** di belakang layar untuk Anda. Sebagai pengingat, jika Anda ingin melihat kode yang dihasilkan, Anda dapat memanggil `tf.autograph.to_code`, dengan memberikan nama fungsi Anda yang dilengkapi dengan properti fungsi Python.

## Urutan Eksekusi dalam Graph

Salah satu hal yang perlu diperhatikan ketika menulis kode yang akan otomatis dikonversi menjadi **graph** adalah memastikan bahwa urutan eksekusi sesuai dengan yang Anda inginkan. Untuk graph sederhana seperti ini, Anda dapat melihat bahwa Anda ingin mengalikan **W** dengan **x** terlebih dahulu, kemudian menambahkan hasilnya ke **b**. 

Penting untuk diingat bahwa ketika menulis Python, instruksi **graph** akan diimplementasikan dalam urutan yang sama dengan kode Python asli. Mari kita lihat contoh berikut.

## Contoh Kode yang Akan Menghasilkan Graph yang Kompleks

Menggunakan `tf.function` untuk secara otomatis menghasilkan **graph** dari kode Python biasa membantu Anda menghindari desain graph yang rumit. Contoh kode yang menghasilkan **graph** rumit adalah ketika variabel yang sama digunakan sebagai input untuk beberapa perhitungan, tetapi mungkin juga digunakan untuk menyimpan hasil dari perhitungan lainnya. Misalnya, pertimbangkan kode ini di mana kita melakukan banyak pembacaan dan penulisan ke variabel **a** dan **b**.

### Penjelasan Eksekusi Fungsi

Hasil dari fungsi `f(1, 2)` adalah 10. Tapi bagaimana cara mencapainya? Kita mulai dengan **a** diinisialisasi menjadi 1.0 dan **b** menjadi 2.0. Hal pertama yang terjadi adalah **a** akan diberi nilai **y** dikali **b**. **B** adalah 2.0, dan **y** adalah parameter kedua dari fungsi, yang juga bernilai 2.0, sehingga **a** sekarang diberi nilai 4.0. 

**B** sudah 2.0, tetapi dengan memanggil fungsi **add**, kita mengalikan **x** (yang bernilai 1) dengan **a**, yang setelah baris sebelumnya menjadi 4, sehingga hasilnya menjadi 4. Ketika ditambahkan ke **b**, maka **b** menjadi 6. Ketika kita mengembalikan **a + b**, kita mendapatkan 10.

Bayangkan bagaimana padatnya **graph** ini dengan semua perubahan penugasan ke **a** dan **b**, dan Anda bisa melihat betapa rumitnya hal ini. Jika Anda mencoba merancang **graph** ini sendiri, akan sangat sulit menghindari kesalahan yang tidak disengaja. Untungnya, **autograph** menangani kompleksitas ini untuk Anda, jadi Anda tidak perlu khawatir.

## Menggunakan Autograph untuk Menghasilkan Graph dengan Kontrol Alur Kondisional

Anda juga dapat menggunakan **autograph** untuk menghasilkan **graph** untuk alur kontrol kondisional. Misalnya, pertimbangkan fungsi sederhana ini di mana jika **x** lebih besar dari 0, kita akan mengembalikan string "positive", jika tidak, kita akan mengembalikan string "negative". 

Berikut adalah kode Python asli yang Anda tulis di sebelah kiri, dan kode **graph** yang akan dihasilkan oleh **autograph** di sebelah kanan. Kita akan mulai dengan melihat bagaimana pemeriksaan apakah **x** lebih besar dari 0 diimplementasikan dalam kode **graph**.

### Implementasi Pemeriksaan Kondisi

Mari kita lihat beberapa baris terakhir dari kode di sebelah kanan. Pertama, ekspresi **x > 0**, yang mengevaluasi menjadi boolean **True** atau **False**, disimpan dalam variabel bernama **cond**. Di bawahnya, objek `ag__` memiliki fungsi bernama **if_statements**, yang menerima beberapa parameter. Kita akan fokus pada tiga parameter pertama.

1. Parameter pertama adalah kondisi boolean.
2. Parameter kedua adalah fungsi yang dipanggil jika kondisi benar.
3. Parameter ketiga adalah fungsi yang dipanggil jika kondisi salah.

### Penggunaan Fungsi di dalam Kondisional

Ekivalen dari perintah `return positive` menjadi sebuah fungsi yang dipanggil **if_true**, yang terlihat di sini. Fungsi ini termasuk pengecekan kesalahan yang dilakukan dalam blok **try except**.

Sekarang ingat, **autograph** telah menghasilkan semua kode ini untuk Anda. Demikian juga, untuk cabang **else** dari pernyataan **if** di kiri, **autograph** menghasilkan fungsi bernama **if_false**. 

### Kesimpulan

Kode di sebelah kiri mudah untuk Anda tulis. Anda hanya perlu menulis kode seperti biasa, dan biarkan **autograph** menghasilkan kode di sebelah kanan untuk Anda. Dengan menggunakan **autograph**, meskipun kode **graph** lebih rumit, Anda dapat menghindari kesalahan dan memfokuskan diri pada hal yang lebih penting.

# **Control Flow in TensorFlow with Autograph**

## **Pendahuluan**
Di bawah ini akan dijelaskan contoh penggunaan `while loop` yang cukup kompleks dalam operasinya. Di dalam loop tersebut, `tf.print(x)` akan mencetak nilai yang tersimpan di dalam `x`. Pada baris berikutnya, `x = tf.tanh(x)` akan mengubah nilai `x` melalui fungsi tanh, kemudian menyimpan hasilnya kembali di `x`. Loop akan terus berlanjut selama jumlah total dari `x` lebih besar dari satu. Jika Anda menjumlahkan semua nilai dalam Tensor `x` dan hasilnya lebih besar dari satu, maka loop akan berlanjut. Fungsi ini berusaha untuk terus menerapkan operasi tanh pada Tensor `x` selama jumlah semua elemennya lebih besar dari satu.

## **Analisis Kode**
Di sisi kiri, kita melihat kode Python biasa, dan di sisi kanan terdapat kode yang dihasilkan oleh Autograph yang ekuivalen. Ini sebenarnya adalah subset kecil dari keseluruhan kode yang dihasilkan oleh Autograph, tetapi sudah mencakup logika yang penting. Mari kita lihat bagaimana kondisi `while` diimplementasikan dalam kode graf. 

Ada fungsi yang disebut `loop_test`, yang memanggil fungsi `converted_call`. Fungsi `converted_call` ini menerima empat parameter fungsi, tetapi kita akan fokus pada dua parameter pertama. Parameter pertama menerima fungsi `tf.reduce_sum`, dan parameter kedua menerima Tensor `x`. Fungsi `converted_call` ini diikuti oleh operator perbandingan `greater than one`, sehingga fungsi `loop_test` akan mengembalikan nilai boolean `True` jika jumlah `x` lebih besar dari satu dan `False` sebaliknya. 

## **Loop Body**
Pada pernyataan berikutnya, fungsi `while` dieksekusi. Fungsi ini menerima enam parameter, dan kita akan fokus pada dua parameter pertama. Parameter pertama menerima fungsi `loop_test`, yang memeriksa apakah loop harus dilanjutkan. Parameter kedua adalah fungsi `loop_body`, yang berisi kode yang akan dijalankan di dalam loop. 

Sekarang, mari kita lihat definisi `loop_body` lebih rinci. Di dalamnya, terdapat beberapa panggilan `converted_call`. Panggilan pertama adalah `tf.print`, yang mencetak nilai `x`. Panggilan berikutnya adalah `tf.tanh`, yang menghitung tanh dari `x`. Perlu diingat bahwa semua kode ini dapat diimplementasikan dengan lebih mudah menggunakan Python biasa di sisi kiri dan Autograph akan menghasilkan kode tersebut untuk Anda.

## **Autograph dan Tracing**
Hal lain yang perlu dipertimbangkan ketika menggunakan Autograph adalah bagaimana peralihan dari kode eksekusi eager ke kode berbasis grafik dapat memengaruhi pernyataan tracing. Pernyataan tracing digunakan untuk membantu pengembang melacak fungsi mana yang telah dijalankan dan nilai variabel yang berubah saat kode berjalan.

Misalnya, pertimbangkan kode berikut di sebelah kiri di mana kita membuat fungsi `f`, yang kemudian dipanggil dengan parameter `x` dan akan mencetak pernyataan bahwa ia dilacak dengan `x`. Fungsi tersebut dipanggil di dalam sebuah loop, seperti yang terlihat di sini, sehingga fungsi akan dipanggil beberapa kali, menghasilkan beberapa cetakan.

Namun, bagaimana jika kita menghias fungsi `f` dengan `tf.function`, yang berarti itu akan beroperasi dalam mode grafik? Juga, perhatikan bahwa sekarang kita menambahkan baris kode lain setelah pernyataan `print`. Kali ini, kita menggunakan pernyataan `tf.print` dari TensorFlow dan memberikan string `executed with` diikuti oleh `x`.

## **Perbedaan antara `print` dan `tf.print`**
Mari kita lihat bagaimana perbedaan antara pernyataan `print` milik Python dan `tf.print` ketika fungsi dihias dengan `tf.function`. Seperti sebelumnya, kita akan memanggil fungsi `f` dari dalam loop sebanyak lima kali. Perhatikan bahwa pernyataan "traced with" hanya dicetak sekali. Hal ini karena pernyataan `print` Python tidak dirancang untuk bekerja di dalam grafik atau sesi yang dijalankan oleh grafik. Python menganggapnya hanya dipanggil sekali. Sebaliknya, `tf.print` dari TensorFlow yang sadar grafik akan mencetak dengan benar dan, seperti yang Anda lihat, mencetak "executed with" dua kali, sebanyak lima kali.

## **Penempatan Variabel dalam Mode Grafik**
Hal lain yang perlu dipertimbangkan adalah di mana Anda membuat variabel Anda. Umumnya, kita berpikir bahwa variabel dapat dibuat di dalam ruang lingkup fungsi sebagai variabel lokal. Tetapi ini dapat menyebabkan kesalahan dalam kode mode grafik, di mana variabel dan operasi harus dipisahkan dan diperlakukan secara terpisah. Memindahkan deklarasi keluar dari fungsi seperti ini akan mengonversinya dengan benar.

## **Latihan**
Latihan minggu ini adalah untuk membangun **classifier kuda atau manusia**. Anda akan menggabungkan banyak konsep baru TensorFlow yang telah Anda pelajari. Salah satu hal yang akan Anda gunakan adalah **TensorFlow Dataset** menggunakan **TFRecord** untuk mendapatkan data. Ketika menggunakan **TFRecord**, Anda sering menulis fungsi pemetaan untuk melakukan pra-pemrosesan pada gambar. Anda akan mengimplementasikan ini sebagai fungsi grafik menggunakan Autograph.

Sebelum melanjutkan ke latihan, saya memiliki sebuah screencast Colab yang berisi sejumlah fungsi yang telah diubah dengan Autograph sehingga Anda dapat melihat kode tersebut. Pastikan untuk melihatnya terlebih dahulu.

---

**Catatan:** Kode ini dapat sangat membantu Anda memahami bagaimana kode grafik bekerja di TensorFlow, serta bagaimana fungsi-fungsi sederhana dapat diubah menjadi bentuk grafis menggunakan Autograph untuk efisiensi yang lebih baik dalam pemrosesan data.